# First AI Agent: Tools and Memory

In this chapter, you will explore AI agents that can decide which tools to use and remember context across turns.

| File | What it demonstrates | GitHub Link |
|------|---------------------|-------------|
| `05_ai_agent_basics_calculator_memory.json` | Agent with Calculator + Memory | [View](https://github.com/ezponda/ai-agents-course/blob/main/courses/n8n_no_code/book/_static/workflows/05_ai_agent_basics_calculator_memory.json) |
| `06_ai_agent_tools_wikipedia_calculator.json` | Agent with Wikipedia + Calculator + Think | [View](https://github.com/ezponda/ai-agents-course/blob/main/courses/n8n_no_code/book/_static/workflows/06_ai_agent_tools_wikipedia_calculator.json) |
| `07_ai_agent_chat_trigger_memory.json` | Chat interface with memory across turns | [View](https://github.com/ezponda/ai-agents-course/blob/main/courses/n8n_no_code/book/_static/workflows/07_ai_agent_chat_trigger_memory.json) |

**Credentials needed:** OpenRouter API key (Settings → Credentials). Wikipedia and Calculator tools need no credentials.

---

## Meet the Node: AI Agent

The **AI Agent** is fundamentally different from the Basic LLM Chain you used in previous chapters.

| Property | Basic LLM Chain | AI Agent |
|----------|-----------------|----------|
| **Execution** | One prompt → one response | Loop: think → act → observe → repeat |
| **Tools** | None | Can call Calculator, Wikipedia, HTTP, etc. |
| **Decision-making** | You control the flow | Agent decides what to do next |
| **Output field** | `text` | `output` |

**Key concept:** The agent can call tools (like Calculator) and decide when it has enough information to answer.

### Sub-Nodes (Connected via Dotted Lines)

The AI Agent uses **sub-nodes** that provide capabilities:

| Sub-node | What it provides | Required? |
|----------|------------------|-----------|
| **Chat Model** | The AI "brain" for reasoning | Yes |
| **Memory** | Remembers conversation history across turns | No |
| **Tools** | Actions the agent can take (Calculator, Wikipedia, etc.) | No |

### Tools You'll Use in This Chapter

| Tool | What it does | Needs credentials? |
|------|--------------|-------------------|
| **Calculator** | Performs arithmetic accurately | No |
| **Wikipedia** | Looks up factual information | No |
| **Think** | A scratchpad for the agent to plan before acting | No |

**See Node Toolbox for full documentation on AI Agent and all tools.**

---

## Example 1: Calculator + Memory

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│  Manual Trigger │────▶│   Edit Fields   │────▶│    AI Agent     │────▶│     Output      │
└─────────────────┘     └─────────────────┘     └─────────────────┘     └─────────────────┘
                                                    ┊ (sub-nodes)
                                                    ┊
                                          ┌─────────┴─────────┐
                                          ┊                   ┊
                                    ┌───────────┐  ┌────────┐  ┌────────────┐
                                    │Chat Model │  │ Memory │  │ Calculator │
                                    └───────────┘  └────────┘  └────────────┘
```

**File:** `05_ai_agent_basics_calculator_memory.json`

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/book/_static/workflows/05_ai_agent_basics_calculator_memory.json
> ```
>
> **Download:** {download}`05_ai_agent_basics_calculator_memory.json <_static/workflows/05_ai_agent_basics_calculator_memory.json>`

::::{dropdown} 🛠️ Build this workflow from scratch (step-by-step)
:color: secondary

### Step 1: Create a new workflow

1. Click **Workflows** → **Add Workflow**
2. Rename it to "AI Agent Calculator Practice"

### Step 2: Add the trigger and input

1. Add **Manual Trigger**
2. Add **Edit Fields** → rename to `Input — Agent Prompt`
3. Add two String fields:

| Name | Value |
|------|-------|
| `chatInput` | `What is 17% tip on 46.50? Use the Calculator tool.` |
| `sessionId` | `demo` |

### Step 3: Add the AI Agent node

1. Add **AI Agent** → rename to `AI Agent — Calculator + Memory`
2. Configure:
   - **Source for Prompt**: `Define below`
   - **Prompt** (Expression): `{{ $json.chatInput }}`
   - **System Message** (in Options):
     ```
     You are a concise assistant.

     Rules:
     - If the user asks for ANY arithmetic, ALWAYS use the Calculator tool.
     - Keep the final answer short (max 5 lines).
     ```

### Step 4: Add the Chat Model (sub-node)

1. Click **+ Chat Model** at the bottom of the AI Agent node
2. Select **OpenRouter Chat Model** (or OpenAI, etc.)
3. Choose your credential
4. Model: `openai/gpt-4o-mini`

### Step 5: Add Memory (sub-node)

1. Click **+ Memory** at the bottom of the AI Agent node
2. Select **Window Buffer Memory**
3. Configure:
   - **Session ID Type**: `Custom Key`
   - **Session Key** (Expression): `{{ $json.sessionId }}`
   - **Context Window Length**: `10`

### Step 6: Add Calculator tool (sub-node)

1. Click **+ Tool** at the bottom of the AI Agent node
2. Select **Calculator**
3. No configuration needed

### Step 7: Add Output

1. Add **Edit Fields** → rename to `Output — Answer`
2. Add field:
   - Name: `answer`
   - Value (Expression): `{{ $json.output }}`

### Step 8: Test

1. Click **Execute Workflow**
2. Check the agent used the Calculator tool (not guessing)
3. Try: "My name is Alex. Remember it." then "What is my name?"

::::

### What Problem This Solves

You want an assistant that can do math reliably (using a Calculator tool instead of guessing) and remember context across multiple runs (using Memory with a session ID).

### Node-by-Node Walkthrough

<div style="overflow: auto; max-height: 200px; border: 1px solid #ddd; border-radius: 4px; padding: 10px; margin-bottom: 15px; background: #f8f8f8;">
<pre style="margin: 0; white-space: pre;">
┌──────────────────┐     ┌────────────────────────┐     ┌────────────────────────────────┐     ┌──────────────────────┐
│   Manual Trigger │────▶│  Input — Agent Prompt  │────▶│  AI Agent — Calculator+Memory  │────▶│  Output — Answer     │
└──────────────────┘     └────────────────────────┘     └────────────────────────────────┘     └──────────────────────┘
                                                                       ┊
                                                                       ┊ (sub-nodes)
                                                        ┌──────────────┼──────────────┐
                                                        ┊              ┊              ┊
                                                 ┌─────────────┐ ┌───────────┐ ┌────────────┐
                                                 │ Chat Model  │ │  Memory   │ │ Calculator │
                                                 └─────────────┘ └───────────┘ └────────────┘
</pre>
</div>

| Node | Type | What it does |
|------|------|-------------|
| **Manual Trigger** | Trigger | Starts the workflow |
| **Input — Agent Prompt** | Set | Creates `chatInput` and `sessionId` fields |
| **AI Agent — Calculator + Memory** | AI Agent | Receives `{{ $json.chatInput }}`, decides to use tools, outputs `output` |
| **Output — Answer** | Set | Saves `{{ $json.output }}` as `answer` |

**Sub-nodes (dotted lines to the Agent):**

| Sub-node | Type | Purpose |
|----------|------|---------|
| **OpenRouter Chat Model** | Chat Model | Provides the AI "brain" |
| **Simple Memory** | Memory | Remembers last 10 messages using `{{ $json.sessionId }}` |
| **Calculator** | Tool | Performs arithmetic when the agent calls it |

### System Message

```
You are a concise assistant.

Rules:
- If the user asks for ANY arithmetic, ALWAYS use the Calculator tool.
- Keep the final answer short (max 5 lines).
```

**Why "ALWAYS use Calculator"?** LLMs are notoriously bad at math. They often guess wrong, especially with percentages or multi-step calculations. Forcing tool use ensures accuracy.

### Default Value Pattern

The Input node uses a useful pattern for testing:

```
{{ $json.chatInput || "What is 17% tip on 46.50? Use the Calculator tool." }}
```

This means: use incoming `chatInput` if it exists, otherwise fall back to a default test value. Helpful when developing workflows.

### Memory Window

The Simple Memory sub-node has `contextWindowLength: 10`, meaning it only remembers the last 10 messages. Older messages are forgotten. This prevents token limits from being exceeded in long conversations.

### Data Flow

```
INPUT                          OUTPUT
─────                          ──────
Trigger: { }
    ↓
Input — Agent Prompt: { chatInput: "What is 17% tip on 46.50?", sessionId: "demo" }
    ↓
AI Agent (thinks → calls Calculator → reads result): { output: "A 17% tip on $46.50 is $7.91." }
    ↓
Output — Answer: { answer: "A 17% tip on $46.50 is $7.91." }
```

### What to Observe

1. Click **Input — Agent Prompt** → see `chatInput` and `sessionId`
2. Click **AI Agent** → see `output` field with the final answer
3. Run again with: `"My name is Alex. Remember it."` → then: `"What is my name?"` (memory works if `sessionId` stays the same)

---

::::{dropdown} 🔍 How does the AI Agent decide to use tools? See the loop
:color: info

Unlike Basic LLM Chain (one call), the AI Agent **loops** until it has enough information:

```
                    ┌─────────────────────────────────────────────┐
                    │              USER QUESTION                  │
                    │  "What is 17% tip on $46.50?"               │
                    └─────────────────────────────────────────────┘
                                        │
                                        ▼
                    ┌─────────────────────────────────────────────┐
                    │           AGENT THINKS (Loop 1)             │
                    │  "I need to calculate 17% of 46.50.         │
                    │   I should use the Calculator tool."        │
                    └─────────────────────────────────────────────┘
                                        │
                                        ▼
                    ┌─────────────────────────────────────────────┐
                    │           CALLS CALCULATOR TOOL             │
                    │  Input: "46.50 * 0.17"                      │
                    │  Output: 7.905                              │
                    └─────────────────────────────────────────────┘
                                        │
                                        ▼
                    ┌─────────────────────────────────────────────┐
                    │           AGENT THINKS (Loop 2)             │
                    │  "Calculator returned 7.905.                │
                    │   I have enough info to answer."            │
                    └─────────────────────────────────────────────┘
                                        │
                                        ▼
                    ┌─────────────────────────────────────────────┐
                    │              FINAL OUTPUT                   │
                    │  { "output": "A 17% tip on $46.50 is        │
                    │               $7.91 (rounded)." }           │
                    └─────────────────────────────────────────────┘
```

**Key insight:** The agent made 2 "thinking" loops: first to decide it needs a tool, then to confirm it has enough info. The Calculator gave an accurate answer instead of the LLM guessing.

::::

## Example 2: Multi-Tool Agent (Wikipedia + Calculator + Think)

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│  Manual Trigger │────▶│   Edit Fields   │────▶│    AI Agent     │────▶│     Output      │
└─────────────────┘     └─────────────────┘     └─────────────────┘     └─────────────────┘
                                                    ┊ (sub-nodes)
                                                    ┊
                                    ┌───────────────┴───────────────┐
                                    ┊               ┊               ┊
                              ┌───────────┐  ┌───────────┐  ┌───────────────────────┐
                              │Chat Model │  │  Memory   │  │ Wikipedia, Calculator │
                              └───────────┘  └───────────┘  │       Think           │
                                                            └───────────────────────┘
```

**File:** `06_ai_agent_tools_wikipedia_calculator.json`

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/book/_static/workflows/06_ai_agent_tools_wikipedia_calculator.json
> ```
>
> **Download:** {download}`06_ai_agent_tools_wikipedia_calculator.json <_static/workflows/06_ai_agent_tools_wikipedia_calculator.json>`

::::{dropdown} 🛠️ Build this workflow from scratch (step-by-step)
:color: secondary

### Step 1: Create a new workflow

1. Click **Workflows** → **Add Workflow**
2. Rename it to "AI Agent Multi-Tool Practice"

### Step 2: Add the trigger and input

1. Add **Manual Trigger**
2. Add **Edit Fields** → rename to `Input — Agent Prompt`
3. Add two String fields:

| Name | Value |
|------|-------|
| `chatInput` | `Using Wikipedia, tell me who Ada Lovelace was (2 bullets). Then give the year of her Notes and compute how many years ago that was from 2026.` |
| `sessionId` | `demo` |

### Step 3: Add the AI Agent node

1. Add **AI Agent** → rename to `AI Agent — Multi-Tool`
2. Configure:
   - **Source for Prompt**: `Define below`
   - **Prompt** (Expression): `{{ $json.chatInput }}`
   - **System Message** (in Options):
     ```
     You are a concise assistant.

     Rules:
     - For factual questions about people/places/dates: verify with Wikipedia.
     - For arithmetic: ALWAYS use Calculator.
     - Use Think briefly (1-2 sentences) to plan before calling tools.
     - Output: short bullets, then the final computed number.
     ```

### Step 4: Add sub-nodes

**Chat Model:**
1. Click **+ Chat Model** → Select **OpenRouter Chat Model**
2. Configure credential and model

**Memory:**
1. Click **+ Memory** → Select **Window Buffer Memory**
2. Session Key: `{{ $json.sessionId }}`, Context Window: `10`

**Tools (add all three):**
1. Click **+ Tool** → Select **Wikipedia** (no config needed)
2. Click **+ Tool** → Select **Calculator** (no config needed)
3. Click **+ Tool** → Select **Think** (no config needed)

### Step 5: Add Output

1. Add **Edit Fields** → rename to `Output — Answer`
2. Add field: `answer` = `{{ $json.output }}`

### Step 6: Test

1. Click **Execute Workflow**
2. Expand execution details to see the tool call sequence
3. Verify: Think → Wikipedia → Calculator → final answer

::::

### New Tool: Think

This example introduces the **Think** tool—different from other tools because it doesn't fetch external data or perform actions.

| Property | Description |
|----------|-------------|
| **Purpose** | A reasoning scratchpad where the agent writes its plan |
| **How it helps** | Explicit planning keeps the agent organized on multi-step tasks |
| **Typical usage** | Agent calls Think before calling other tools |

**Example Think output:** "I need to look up Ada Lovelace on Wikipedia first, then calculate years since 1843."

Without Think, agents sometimes jump straight to tools and get confused about the order of operations.

### What Problem This Solves

You want an agent that can look up facts (Wikipedia), do math (Calculator), and plan before acting (Think tool). The agent decides which tool to use based on the question.

### Node-by-Node Walkthrough

<div style="overflow: auto; max-height: 220px; border: 1px solid #ddd; border-radius: 4px; padding: 10px; margin-bottom: 15px; background: #f8f8f8;">
<pre style="margin: 0; white-space: pre;">
┌──────────────────┐     ┌────────────────────────┐     ┌────────────────────────┐     ┌──────────────────────┐
│   Manual Trigger │────▶│  Input — Agent Prompt  │────▶│  AI Agent — Multi-Tool │────▶│  Output — Answer     │
└──────────────────┘     └────────────────────────┘     └────────────────────────┘     └──────────────────────┘
                                                                    ┊
                                                                    ┊ (sub-nodes)
                                                 ┌──────────────────┼──────────────────┐
                                                 ┊                  ┊                  ┊
                                          ┌─────────────┐    ┌───────────┐    ┌─────────────────────────────┐
                                          │ Chat Model  │    │  Memory   │    │ Wikipedia, Calculator, Think│
                                          └─────────────┘    └───────────┘    └─────────────────────────────┘
</pre>
</div>

| Node | Type | What it does |
|------|------|-------------|
| **Manual Trigger** | Trigger | Starts the workflow |
| **Input — Agent Prompt** | Set | Creates `chatInput` and `sessionId` fields |
| **AI Agent — Multi-Tool** | AI Agent | Receives `{{ $json.chatInput }}`, chooses tools, outputs `output` |
| **Output — Answer** | Set | Saves `{{ $json.output }}` as `answer` |

**Sub-nodes (dotted lines to the Agent):**

| Sub-node | Type | Purpose |
|----------|------|---------|
| **OpenRouter Chat Model** | Chat Model | Provides the AI "brain" |
| **Simple Memory** | Memory | Remembers last 10 messages using `{{ $json.sessionId }}` |
| **Wikipedia** | Tool | Looks up factual information (no API key needed) |
| **Calculator** | Tool | Performs arithmetic |
| **Think** | Tool | A "scratchpad" where the agent writes its reasoning |

### System Message

```
You are a concise assistant.

Rules:
- For factual questions about people/places/dates: verify with Wikipedia.
- For arithmetic: ALWAYS use Calculator.
- Use Think briefly (1-2 sentences) to plan before calling tools.
- Output: short bullets, then the final computed number.
```

### Example Default Prompt

The workflow's default test question demonstrates multi-tool orchestration:

```
Using Wikipedia, tell me who Ada Lovelace was (2 bullets). 
Then give the year of her Notes and compute how many years ago that was from 2026.
```

This requires: Think (plan) → Wikipedia (facts) → Calculator (math) → format answer.

### Data Flow

```
INPUT                          OUTPUT
─────                          ──────
Trigger: { }
    ↓
Input — Agent Prompt: { chatInput: "Using Wikipedia, tell me who Ada Lovelace was...", sessionId: "demo" }
    ↓
AI Agent:
  1. Calls Think: "I'll search Wikipedia for Ada Lovelace, then calculate..."
  2. Calls Wikipedia: gets biography
  3. Calls Calculator: 2026 - 1843 = 183
  4. Formats final answer
    ↓
Output — Answer: { answer: "• Ada Lovelace was a mathematician... • 183 years ago" }
```

### What to Observe

1. Click **AI Agent — Multi-Tool** after execution
2. In the Output panel, expand the execution details to see tool calls in order
3. Look for the Think tool output—it shows the agent's planning step
4. Notice how Wikipedia is called before Calculator (the agent figured out the right order)

---

::::{dropdown} 🔍 How does the agent orchestrate multiple tools? See the sequence
:color: info

For complex questions, the agent chains multiple tools in the right order:

```
┌─────────────────────────────────────────────────────────────────────────────────┐
│  QUESTION: "Who was Ada Lovelace? What year were her Notes published?           │
│             How many years ago was that from 2026?"                             │
└─────────────────────────────────────────────────────────────────────────────────┘
                                        │
                                        ▼
┌─────────────────────────────────────────────────────────────────────────────────┐
│  STEP 1: THINK TOOL                                                             │
│  ─────────────────                                                              │
│  Agent writes: "I need to:                                                      │
│    1. Look up Ada Lovelace on Wikipedia                                         │
│    2. Find the year of her Notes                                                │
│    3. Calculate 2026 minus that year"                                           │
└─────────────────────────────────────────────────────────────────────────────────┘
                                        │
                                        ▼
┌─────────────────────────────────────────────────────────────────────────────────┐
│  STEP 2: WIKIPEDIA TOOL                                                         │
│  ──────────────────────                                                         │
│  Query: "Ada Lovelace"                                                          │
│  Result: "Augusta Ada King, Countess of Lovelace (1815-1852) was an English     │
│           mathematician. Her notes on Babbage's Analytical Engine (1843)        │
│           are considered the first computer program..."                         │
└─────────────────────────────────────────────────────────────────────────────────┘
                                        │
                                        ▼
┌─────────────────────────────────────────────────────────────────────────────────┐
│  STEP 3: CALCULATOR TOOL                                                        │
│  ───────────────────────                                                        │
│  Input: "2026 - 1843"                                                           │
│  Result: 183                                                                    │
└─────────────────────────────────────────────────────────────────────────────────┘
                                        │
                                        ▼
┌─────────────────────────────────────────────────────────────────────────────────┐
│  FINAL OUTPUT                                                                   │
│  ────────────                                                                   │
│  {                                                                              │
│    "output": "• Ada Lovelace was an English mathematician (1815-1852)           │
│               • Her Notes on the Analytical Engine were published in 1843       │
│               • That was 183 years ago from 2026"                               │
│  }                                                                              │
└─────────────────────────────────────────────────────────────────────────────────┘
```

**Key insight:** The Think tool helped the agent plan the sequence. Without it, agents sometimes call Wikipedia multiple times or forget to do the calculation.

::::

## Example 3: Chat Trigger + Memory

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│  Chat Trigger   │────▶│    AI Agent     │────▶│     Output      │
└─────────────────┘     └─────────────────┘     └─────────────────┘
                            ┊ (sub-nodes)
                            ┊
                  ┌─────────┴─────────┐
                  ┊                   ┊
            ┌───────────┐  ┌────────┐  ┌────────────┐
            │Chat Model │  │ Memory │  │ Calculator │
            └───────────┘  └────────┘  └────────────┘
```

**File:** `07_ai_agent_chat_trigger_memory.json`

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/book/_static/workflows/07_ai_agent_chat_trigger_memory.json
> ```
>
> **Download:** {download}`07_ai_agent_chat_trigger_memory.json <_static/workflows/07_ai_agent_chat_trigger_memory.json>`

::::{dropdown} 🛠️ Build this workflow from scratch (step-by-step)
:color: secondary

### Step 1: Create a new workflow

1. Click **Workflows** → **Add Workflow**
2. Rename it to "Chat Agent Practice"

### Step 2: Add the Chat Trigger

1. Add **Chat Trigger** (search for "When chat message received")
2. Configure:
   - **Mode**: `Webhook` (default)
   - **Response Mode**: `When Last Node Finishes` (look for `responseMode`)

**Important:** The Chat Trigger outputs `chatInput` and `sessionId` automatically.

### Step 3: Add the AI Agent node

1. Add **AI Agent** → rename to `AI Agent — Chat + Memory + Calculator`
2. Configure:
   - **Source for Prompt**: `Define below`
   - **Prompt** (Expression): `{{ $json.chatInput }}`
   - **System Message** (in Options):
     ```
     You are a helpful assistant inside an n8n AI Agent workflow. 
     Be concise. 
     If the user asks for arithmetic, use the Calculator tool. 
     If the user asks you to remember something, store it in memory and confirm briefly. 
     If you don't know, say so.
     ```

### Step 4: Add sub-nodes

**Chat Model:**
1. Click **+ Chat Model** → Select **OpenRouter Chat Model**
2. Configure credential and model

**Memory:**
1. Click **+ Memory** → Select **Window Buffer Memory**
2. Session Key: `{{ $json.sessionId }}`, Context Window: `10`

**Calculator:**
1. Click **+ Tool** → Select **Calculator**

### Step 5: Add Output (IMPORTANT: field must be named `output`)

1. Add **Edit Fields** → rename to `Output — Chat Response`
2. Add field:
   - Name: **`output`** (not `answer`!)
   - Value (Expression): `{{ $json.output }}`

**Why `output`?** The Chat Trigger looks for a field named `output` in the last node. Other names show raw JSON.

### Step 6: Test

1. Click **Open chat** button (not "Execute Workflow")
2. Type: "My name is Alex. Remember it."
3. Type: "What is my name?" → should remember
4. Type: "Calculate 19 * 27" → should use Calculator

::::

### Meet the Node: Chat Trigger

This example introduces a new trigger: **Chat Trigger** (instead of Manual Trigger).

| Property | Description |
|----------|-------------|
| **Purpose** | Provides a chat interface in n8n |
| **How to use** | Click the **Open chat** button (not "Execute Workflow") |
| **Outputs** | `chatInput` (user message) and `sessionId` (conversation ID) |

**Important configuration:**

| Setting | Value | Why it matters |
|---------|-------|----------------|
| `mode` | `webhook` | Listens for incoming chat messages |
| `responseMode` | `lastNode` | Returns whatever the last node outputs |

**Critical:** When `responseMode` is `lastNode`, the Chat Trigger looks for a field named **`output`** in the last node. If you use a different name (like `answer`), the chat shows raw JSON instead of the response text.

**See the **Node Toolbox** appendix for full documentation.**

### What Problem This Solves

You want a real chat interface (not manual test runs) where the agent remembers the conversation across multiple messages within the same session.

### Node-by-Node Walkthrough

<div style="overflow: auto; max-height: 200px; border: 1px solid #ddd; border-radius: 4px; padding: 10px; margin-bottom: 15px; background: #f8f8f8;">
<pre style="margin: 0; white-space: pre;">
┌───────────────────────────────┐     ┌───────────────────────────────────┐     ┌──────────────────────────┐
│  When chat message received   │────▶│  AI Agent — Chat+Memory+Calc      │────▶│  Output — Chat Response  │
└───────────────────────────────┘     └───────────────────────────────────┘     └──────────────────────────┘
                                                        ┊
                                                        ┊ (sub-nodes)
                                         ┌──────────────┼──────────────┐
                                         ┊              ┊              ┊
                                  ┌─────────────┐ ┌───────────┐ ┌────────────┐
                                  │ Chat Model  │ │  Memory   │ │ Calculator │
                                  └─────────────┘ └───────────┘ └────────────┘
</pre>
</div>

| Node | Type | What it does |
|------|------|-------------|
| **When chat message received** | Chat Trigger | Provides chat UI, outputs `chatInput` and `sessionId` |
| **AI Agent — Chat + Memory + Calculator** | AI Agent | Receives `{{ $json.chatInput }}`, uses memory & tools, outputs `output` |
| **Output — Chat Response** | Set | Saves `{{ $json.output }}` as `output` (required name for chat) |

**Sub-nodes (dotted lines to the Agent):**

| Sub-node | Type | Purpose |
|----------|------|---------|
| **OpenRouter Chat Model** | Chat Model | Provides the AI "brain" |
| **Simple Memory (10 messages)** | Memory | Remembers last 10 messages using `{{ $json.sessionId }}` |
| **Calculator** | Tool | Performs arithmetic |

### System Message

```
You are a helpful assistant inside an n8n AI Agent workflow. 
Be concise. 
If the user asks for arithmetic, use the Calculator tool. 
If the user asks you to remember something, store it in memory and confirm briefly. 
If you don't know, say so.
```

### Why the Output Field Must Be Named `output`

This is the most common mistake with Chat Trigger workflows:

| Last node field | Chat UI shows |
|-----------------|---------------|
| `output: "Hello!"` | Hello! |
| `answer: "Hello!"` | `{"answer": "Hello!"}` (raw JSON) |
| `response: "Hello!"` | `{"response": "Hello!"}` (raw JSON) |

The Chat Trigger specifically looks for a field named `output`. That's why the Output node in this workflow saves to `output`, not `answer` like the other examples.

### Data Flow

```
INPUT (from Chat UI)           OUTPUT (to Chat UI)
────────────────────           ───────────────────
Chat Trigger: { chatInput: "My name is Alex.", sessionId: "abc123" }
    ↓
AI Agent: { output: "Got it! I'll remember your name is Alex." }
    ↓
Output — Chat Response: { output: "Got it! I'll remember your name is Alex." }
                              ↑
                     This field name matters!

(next message in same session)
Chat Trigger: { chatInput: "What is my name?", sessionId: "abc123" }
    ↓
AI Agent (reads memory): { output: "Your name is Alex." }
```

### How to Use

1. Import the workflow
2. Click the **Open chat** button (not "Execute Workflow"):

![Open chat button](../images/chat_button.png)

3. Type messages in the chat interface
4. Memory persists within the same chat session

### What to Observe

1. Send: `"My name is Alex. Remember it."`
2. Send: `"What is my name?"` → Agent should recall "Alex"
3. Send: `"Compute 19 * 27"` → Agent uses Calculator tool
4. Open a new chat session → Memory resets (different `sessionId`)

---

::::{dropdown} 🔍 How does Memory work across chat messages? See the session flow
:color: info

Memory uses `sessionId` to keep conversations separate. Same session = shared memory:

```
┌─────────────────────────────────────────────────────────────────────────────────┐
│                            CHAT SESSION A (sessionId: "abc123")                 │
└─────────────────────────────────────────────────────────────────────────────────┘

  MESSAGE 1                           MESSAGE 2                         MESSAGE 3
┌─────────────────┐                ┌─────────────────┐               ┌─────────────────┐
│ User: "My name  │                │ User: "What is  │               │ User: "Calculate│
│  is Alex"       │                │  my name?"      │               │  15 * 8"        │
└─────────────────┘                └─────────────────┘               └─────────────────┘
        │                                  │                                 │
        ▼                                  ▼                                 ▼
┌─────────────────┐                ┌─────────────────┐               ┌─────────────────┐
│ Memory: [ ]     │                │ Memory:         │               │ Memory:         │
│ (empty)         │                │ ["My name is    │               │ ["My name is    │
│                 │                │   Alex",        │               │   Alex",        │
│                 │                │  "Got it!..."]  │               │  "Got it!...",  │
│                 │                │                 │               │  "What is my    │
│                 │                │                 │               │   name?",       │
│                 │                │                 │               │  "Your name is  │
│                 │                │                 │               │   Alex"]        │
└─────────────────┘                └─────────────────┘               └─────────────────┘
        │                                  │                                 │
        ▼                                  ▼                                 ▼
┌─────────────────┐                ┌─────────────────┐               ┌─────────────────┐
│ Agent: "Got it! │                │ Agent: "Your    │               │ Agent: "15 * 8  │
│  I'll remember  │                │  name is Alex." │               │  = 120"         │
│  your name is   │                │                 │               │ (uses Calculator│
│  Alex."         │                │ ✅ Remembered!  │               │  tool)          │
└─────────────────┘                └─────────────────┘               └─────────────────┘


┌─────────────────────────────────────────────────────────────────────────────────┐
│                            CHAT SESSION B (sessionId: "xyz789")                 │
│                                   (NEW WINDOW)                                  │
└─────────────────────────────────────────────────────────────────────────────────┘

  MESSAGE 1
┌─────────────────┐
│ User: "What is  │
│  my name?"      │
└─────────────────┘
        │
        ▼
┌─────────────────┐
│ Memory: [ ]     │
│ (empty - new    │
│  session!)      │
└─────────────────┘
        │
        ▼
┌─────────────────┐
│ Agent: "I don't │
│  know your name.│
│  You haven't    │
│  told me yet."  │
│                 │
│ ❌ No memory    │
│    of Alex!     │
└─────────────────┘
```

**Key insight:** Opening a new chat window creates a new `sessionId`, so the agent starts fresh with no memory of previous conversations.

::::

## The Agent Loop

When an AI Agent runs, it follows this pattern:

1. **Read** the question and system message
2. **Think:** "Do I need a tool?"
3. **If yes:** Call a tool, read its result
4. **Think:** "Do I have enough info now?"
5. **Repeat** until confident, then output the final answer

This loop is why agents are powerful—they can handle complex questions that require multiple steps or tools.

---

## Memory and Session IDs

Memory uses a **Session ID** to keep conversations separate:

| Session ID | Effect |
|------------|--------|
| Same `sessionId` | Same conversation thread (memory persists) |
| Different `sessionId` | Fresh start (no memory) |

In the example workflows, memory uses `{{ $json.sessionId }}` from the input. This means:
- In Manual Trigger workflows: you control the session ID in the Set node
- In Chat Trigger workflows: n8n generates a unique session ID for each chat window

---

## System Messages Control Agent Behavior

The **System Message** in the AI Agent node defines rules for the agent. Examples from the workflows:

**Workflow 05 (Calculator + Memory):**
```
You are a concise assistant.

Rules:
- If the user asks for ANY arithmetic, ALWAYS use the Calculator tool.
- Keep the final answer short (max 5 lines).
```

**Workflow 06 (Multi-Tool):**
```
You are a concise assistant.

Rules:
- For factual questions about people/places/dates: verify with Wikipedia.
- For arithmetic: ALWAYS use Calculator.
- Use Think briefly (1-2 sentences) to plan before calling tools.
```

Good system messages:
- Tell the agent **when** to use each tool
- Set **boundaries** (e.g., "max 5 lines")
- Prevent loops (e.g., "search once, then answer")

---

## Guardrails: Preventing Runaway Agents

Agents loop until they decide they're done. Without limits, a confused agent can loop forever — burning through your API credits. **Always set guardrails.**

### Max Iterations (Critical)

The AI Agent node has a **Max Iterations** setting that limits how many think-act cycles it can perform:

| Setting | Location | Recommended |
|---------|----------|-------------|
| **Max Iterations** | AI Agent → Settings (gear icon) → Max Iterations | 5-10 for most tasks |

**What happens when reached:** The agent stops and returns whatever it has so far, with a note that it hit the limit.

**Cost perspective:** Each iteration = 1-2 API calls. At ~$0.01-0.03 per call (GPT-4o), an agent stuck in a loop for 100 iterations could cost $1-3. Without any limit, it could run indefinitely.

### System Message Boundaries

In addition to Max Iterations, add explicit limits in your system message:

```
Rules:
- Search Wikipedia at most ONCE per question.
- If Calculator returns an error, report it — do not retry.
- Do not call more than 3 tools total per question.
```

### Warning Signs of a Looping Agent

Watch for these in the Output panel:
- Same tool called repeatedly with identical inputs
- Agent keeps saying "Let me verify..." or "Let me check again..."
- Execution time exceeds 30 seconds for a simple question

---

## Debugging: Understanding What the Agent Thinks

When an agent gives wrong answers or behaves unexpectedly, you need to see its "thought process." Here's how.

### Reading the Output Panel

After running the workflow, click the **AI Agent** node and examine the Output panel:

1. **Input tab:** Shows what the agent received (`chatInput`, `sessionId`)
2. **Output tab:** Shows the final `output` field
3. **Execution details:** Click to expand and see **every tool call**

### What to Look For

| Symptom | Where to check | Common cause |
|---------|---------------|--------------|
| Wrong answer | Tool call outputs | Tool returned unexpected data |
| Skipped a tool | System message | Rules not explicit enough |
| Used wrong tool | Agent's reasoning | Ambiguous question |
| Hallucinated facts | Tool calls section | Agent didn't call Wikipedia when it should |

### Example Debug Session

**Problem:** Agent says "Paris has 5 million people" without checking Wikipedia.

**Debug steps:**
1. Click AI Agent node → Output panel
2. Expand execution details
3. Look for Wikipedia tool call → **Not found!**
4. **Diagnosis:** Agent used its training data instead of the tool
5. **Fix:** Update system message: "For ANY factual question about places, ALWAYS use Wikipedia first."

### The "Think" Tool for Transparency

Adding the **Think** tool (see Example 2) makes debugging easier because the agent writes its plan before acting. You can see exactly why it chose each tool.

---

## Common Issues

### Agent Keeps Calling Tools Forever

**Fix:** Set **Max Iterations** (see Guardrails above) and add boundaries to the system message:
```
Search once for current information, then provide your answer.
Do not search more than twice.
```

### Agent Never Uses Tools

**Fix:** Ask questions that clearly need external data, or make the system message more explicit about when to use tools.

### Chat Response Shows Raw JSON

**Fix:** Ensure the last node outputs a field named `output` (not `answer` or another name). The Chat Trigger expects `output`.

### Memory Doesn't Work

**Fix:** Check that `sessionId` is the same across runs. Different session IDs = different memory contexts.

## Summary

| Concept | What You Learned |
|---------|------------------|
| **AI Agent node** | Loops until it has enough info to answer |
| **Chat Model (sub-node)** | Provides reasoning capability |
| **Tools (sub-nodes)** | Actions the agent can call (Calculator, Wikipedia, Think) |
| **Memory (sub-node)** | Enables conversation continuity via `sessionId` |
| **System Message** | Controls when the agent uses tools and how it responds |
| **Output field** | AI Agent outputs to `output` (not `text` like Basic LLM Chain) |

**Key expression:** `{{ $json.output }}` — how you access the agent's response in the next node.

**Docs:** [AI Agent Node](https://docs.n8n.io/integrations/builtin/cluster-nodes/root-nodes/n8n-nodes-langchain.agent/)